In [5]:
import os

In [6]:
#############################################################################################################
#   Create .config, .submit files -> change to your own folder
#
submit_dir = '/home/tchau/code/condor_submit/UL'
logdir = '/scratch/tchau/DM_OscNext/MakePDF/logs/'
if not (os.path.exists(submit_dir)): os.makedirs(submit_dir)
if not (os.path.exists(logdir)): os.makedirs(logdir)


cfig_file = f'{submit_dir}/dagman.config'
sub_file = f'{submit_dir}/UL.submit'
curdir=os.getcwd()
script = f'{curdir}/UpperLimit_LLH_Interval.py'

with open(cfig_file, 'w') as c:
    c.write('DAGMAN_MAX_JOBS_SUBMITTED=100'+'\n')
    c.write('DAGMAN_MAX_SUBMIT_PER_INTERVAL=20'+'\n')
    c.write('DAGMAN_USER_LOG_SCAN_INTERVAL=10'+'\n')
    c.close()


with open(sub_file, 'w') as s:
    s.write(f'executable = {script}'+'\n')
    s.write('\n')
    s.write('initialdir = /data/user/tchau/'+'\n')
    s.write(f'logdir = {logdir}'+'\n')
    s.write('output = $(logdir)$(JOBNAME).$(Cluster).out'+'\n')
    s.write('error = $(logdir)$(JOBNAME).$(Cluster).err'+'\n')
    s.write('log = $(logdir)$(JOBNAME).$(Cluster).log'+'\n')
    s.write('\n')
    s.write('notification   = never'+'\n')
    s.write('universe       = vanilla'+'\n')
    s.write('\n')
    s.write('should_transfer_files = YES'+'\n')
    s.write('request_memory = 6GB'+'\n')
    s.write('getenv = True'+'\n')
    s.write('\n')
    s.write('arguments = -c $(c) -p $(p) --process $(proc)  -u $(u) -l $(l) -n $(n) --mc $(mc) -b $(bkg) --nsample $(nsample) --errorJ $(errorJ) --gcinj $(gcinj) --gcmodel $(gcmodel)'+'\n')
    s.write('queue')
    s.close()

In [8]:
#############################################################################################################
#   Create .dag and submit script
#
submit = f'{submit_dir}/submit.sh'
dag_file = '{}/UL.dag'.format(submit_dir)


#list of channels and profiles
channels = ["numunumu"]
# channels = ["WW","bb","tautau","mumu","nuenue","numunumu","nutaunutau"]
profile = ['NFW']
# profile = ['NFW','Burkert']


# process
prcs = ["ann", "decay"]

# Jfactor
errorJ = 'nominal' #error1, error2

# number of mass scan point
n = 30

# number of trials for Brazillian bands (0: no trials)
nsample = 0

# Load a precomputed Background 
bkgs=['precomp']

# MC sample
mcs = ['1122']
# mcs = ['0000', '1124', '1125', '1126', '1127', '1128', '1129', '1130', '1131', '1132', '1133', '1122_disCSMS', '1122_disCSMSx3', '1122_disCSMSx-3']
# mcs = ['1122_disCSMS', '1122_disCSMSx3', '1122_disCSMSx-3']

# GP injection and in the LLR
# ['None','pi0', 'pi0_IC', 'KRA50', 'KRA50_IC']

gcinj = 'None'
gcmodel = 'None'

# Loops on scenarios and submit jobs for each of them
with open(dag_file, 'w') as f:
    for bkg in bkgs:
        for mc in mcs:
            for c in channels:
                for p in profile:
                    for prc in prcs:
                        if prc=='ann':
                            masses = {"WW":[90, 8000], "bb":[15, 8000], 'tautau':[5, 4000], 'mumu':[5, 1000], "nuenue":[5, 200],"numunumu":[5, 200],"nutaunutau":[5,200]}
                        elif prc=='decay':    
                            masses = {"WW":[180, 8000], "bb":[30, 8000], 'tautau':[5, 8000], 'mumu':[5, 2000], "nuenue":[5, 400],"numunumu":[5, 400],"nutaunutau":[5,400]}
                        # masses = {"WW":[180, 8000], "bb":[30, 8000], 'tautau':[5, 8000], 'mumu':[5, 2000], "nuenue":[1.5, 5],"numunumu":[1.5, 5],"nutaunutau":[1.5,5]}

                            f.write('JOB UL_{}_{}_{}_{}_{}_bkg{}_{}trial_Jf_{}'.format(prc, c,p, n, mc, bkg, nsample, errorJ)+' UL.submit \n')
                            f.write('VARS UL_{}_{}_{}_{}_{}_bkg{}_{}trial_Jf_{}'.format(prc, c , p, n, mc, bkg, nsample, errorJ)
                                    +' JOBNAME="UL_{}_{}_{}_{}_{}_bkg{}_{}trial_Jf_{}"'.format(prc, c, p, n, mc, bkg, nsample, errorJ) 
                                    +' c="{}" p="{}" proc="{}" u="{}" l="{}" n="{}" mc="{}" bkg="{}" nsample="{}" errorJ="{}" gcinj="{}" gcmodel="{}"'.format(c, p, prc, masses[c][1], masses[c][0], n, mc, bkg, nsample, errorJ, gcinj, gcmodel)+'\n')
f.close()
with open(submit, 'w') as s:
    s.write("#!/bin/sh\n")
    s.write("condor_submit_dag -config dagman.config {}\n".format(dag_file))
    print("condor_submit_dag -config dagman.config {}\n".format(dag_file))
s.close()


condor_submit_dag -config dagman.config /home/tchau/code/condor_submit/UL/UL.dag

